In [78]:
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict
import pandas as pd
from copy import deepcopy
from pathlib import Path
import matplotlib.pylab as plt
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import math
import os
from PIL import Image

In [209]:
csv_path = 'df129_1_tr.csv'
df_TR_1 = pd.read_csv(csv_path)

csv_path_min = 'df129_2_tr_min_values.csv'
df_TR_1_min = pd.read_csv(csv_path_min)

csv_path_GT = '129_1_clean.txt'
df_GT_1 = pd.read_csv(csv_path_GT)

csv_path_GT2 = '129_2_clean.txt'
df_GT_2 = pd.read_csv(csv_path_GT2)

csv_path_TR_2 = 'df129_2_tr.csv'
df_TR_2 = pd.read_csv(csv_path_TR_2)

# csv_path_TR_1 = 'df129_1_tr.csv'
# df_TR_1 = pd.read_csv(csv_path_TR_1)

# csv_path_TR_349_1 = 'df349_1_tr.csv'
# df_TR_349_1 = pd.read_csv(csv_path_TR_349_1 )

# csv_path_GT_349_1 = '349_1_clean.txt'
# df_GT_349_1 = pd.read_csv(csv_path_GT_349_1)

# csv_path_TR_349_2 = 'df349_2_tr.csv'
# df_TR_349_2 = pd.read_csv(csv_path_TR_349_2 )

# csv_path_GT_349_2 = '349_2_clean.txt'
# df_GT_349_2 = pd.read_csv(csv_path_GT_349_2)

display(df_GT_1.loc[df_GT_1['frame'] == 1])
# display(df_GT_2)
# display(df_GT_349_1)
# display(df_GT_349_2)
display(df_TR_1.loc[df_TR_1['frame'] == 1])
display(df_TR_1_min.loc[df_TR_1_min['frame'] == 1])
# display(df_TR_2)
# display(df_TR_349_1)
# display(df_TR_349_2)

,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
5,1,6,1211.27,319.51,27.47,8.96
6,1,7,1069.29,499.23,20.90,10.75
7,1,8,1091.68,476.24,17.32,7.77
8,1,9,1190.41,591.30,32.61,9.32


,frame,id,x,y,x-offset,y-offset
7,1,1.0,1226.222900,423.246933,12.124878,7.287979
8,1,2.0,1495.225037,441.589981,8.940613,10.255325
9,1,3.0,1205.950378,595.077942,15.249817,5.330994
10,1,4.0,1078.227783,504.820404,10.332153,5.378937
11,1,5.0,1222.935974,323.637695,11.778259,5.087555
12,1,6.0,1097.038940,478.523941,9.662964,3.970474
13,1,7.0,1162.937012,419.468430,13.342285,4.543686


,frame,id,x,y,x-offset,y-offset
9,1,1.0,1150.323730,338.193634,10.634521,7.357986
10,1,2.0,1104.555176,442.054138,12.951355,5.719864
11,1,3.0,1038.860840,438.374390,17.207092,6.583496
12,1,4.0,1447.776123,462.898041,9.127991,10.064865
13,1,5.0,1009.387695,357.384003,10.366760,4.096405
14,1,6.0,1173.229004,340.945068,6.377808,6.237946
15,1,7.0,988.105164,529.225891,9.204071,4.042847
16,1,8.0,978.081238,500.251892,12.376068,4.137070
17,1,9.0,1069.182373,626.775208,21.659058,3.357788


In [210]:
# df_TT = df_TR_1.copy()

In [211]:
def find_id_transitions(df, frame_threshold=100):
    # Dictionary to store the last position of each ID
    last_positions = {}

    # Set to store unique ID transitions
    unique_id_transitions = set()

    for index, row in df.iterrows():
        frame = row['frame']
        id_value = row['id']
        x = row['x']
        y = row['y']

        if id_value in last_positions:
            # If ID already exists in last_positions
            last_frame, last_x, last_y = last_positions[id_value]

            if frame - last_frame > frame_threshold:
                # ID has disappeared for more than frame_threshold frames
                last_positions.pop(id_value)
        else:
            # ID is not in last_positions
            for prev_id, (prev_frame, prev_x, prev_y) in last_positions.items():
                distance = ((x - prev_x) ** 2 + (y - prev_y) ** 2) ** 0.5
                if distance < 50 and frame - prev_frame < frame_threshold:
                    # New ID appeared close to the position of a disappeared ID
                    # Check if this transition has already been found
                    if (prev_id, id_value) not in unique_id_transitions:
                        unique_id_transitions.add((prev_id, id_value))

        # Update last position for the current ID
        last_positions[id_value] = (frame, x, y)

    return list(unique_id_transitions)
    
def filter_tuples_by_first_value(tuples_list, values_list):
    filtered_tuples = []
    for pair in tuples_list:
        first_value, second_value = pair
        if first_value in values_list and second_value not in values_list:
            filtered_tuples.append(pair)
    return filtered_tuples

def remove_tuples_with_large_time_gap(tuples_list, df):
    filtered_tuples = []
    for pair in tuples_list:
        first_id, second_id = pair
        first_id_last_frame = df[df['id'] == first_id]['frame'].max()
        second_id_first_frame = df[df['id'] == second_id]['frame'].min()
        
        if abs(first_id_last_frame - second_id_first_frame) < 100 :
            filtered_tuples.append(pair)
    
    return filtered_tuples

def replace_ids_in_dataframe(tuples_list, df):
    for pair in tuples_list:
        first_id, second_id = pair
        df.loc[df['id'] == second_id, 'id'] = first_id
    
    return df


def renumber_ids(df):
    # Get unique IDs in the DataFrame
    unique_ids = df['id'].unique()
    
    # Create a mapping from old IDs to new IDs
    id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_ids)}
    
    # Replace old IDs with new IDs in the DataFrame
    df['id'] = df['id'].map(id_mapping)
    
    return df

In [212]:
def Re_ID(df):
    
#     Step 1
    less_than_5_list = []

    # Group by 'ID' and count unique frames each ID appears in
    id_counts = df.groupby('id')['frame'].nunique()

    # Filter IDs that appear in less than 5 frames
    ids_less_than_5_frames = id_counts[id_counts < 30]

    # Create a dictionary to store the corresponding frames for each ID
    id_frames_dict = {}

    # Iterate over the IDs and find the corresponding frames
    for id_ in ids_less_than_5_frames.index:
        frames = df[df['id'] == id_]['frame'].unique().tolist()
        id_frames_dict[id_] = frames

    for id_, frames in id_frames_dict.items():
        less_than_5_list.append(id_)

    # List of IDs to remove
    ids_to_remove = less_than_5_list

    # Removing rows with IDs from the list
    df = df[~df['id'].isin(ids_to_remove)]
    
#     Step 2
    
    begin_ids = list(df['id'].loc[df['frame'] == 0])

    # Given list of IDs
    given_ids = begin_ids

    not_untill_end = []

    last_frame_appearance = {}

    # Iterate over the IDs and find the last frame where each ID appears
    for id in given_ids:
        last_frame = df[df['id'] == id]['frame'].max()
        last_frame_appearance[id] = last_frame if not pd.isnull(last_frame) else "Does not appear"
        if last_frame != df['frame'].max():
            not_untill_end.append(id)

    id_transitions = find_id_transitions(df, frame_threshold=100)

    filtered_tuples = filter_tuples_by_first_value(id_transitions, not_untill_end)
    
#     Step 3
    
    # check if second id in tuple appears 40 frames before last frame of first id in tuple, 
    # if so then probably just two fish close to each other

    filtered_tuples = remove_tuples_with_large_time_gap(filtered_tuples, df)
    
#     Step 4


    # Create a dictionary where keys are the second elements and values are lists of corresponding first elements
    second_to_first_dict = {}
    for first, second in filtered_tuples:
        if second not in second_to_first_dict:
            second_to_first_dict[second] = [first]
        else:
            second_to_first_dict[second].append(first)

    # Check if any value has more than one occurrence
    for second, first_list in second_to_first_dict.items():
        if len(first_list) > 1:
            for first in first_list:
                # Check if the corresponding first element appears in another tuple
                if any((first, other_second) in filtered_tuples for other_second in second_to_first_dict if other_second != second):
#                     print("Tuple with second element", second, "and first element", first, "meets the condition")

                    # Tuple to remove
                    tuple_to_remove = (first, second)

                    # Remove the tuple from the tuple list
                    filtered_tuples = [pair for pair in filtered_tuples if pair != tuple_to_remove]

#     Step 5

    ids_to_remove = [x[1] for x in filtered_tuples]
    
    df = replace_ids_in_dataframe(filtered_tuples, df)

#     Step 6
    # check for ids from list that dissapear and for ids that appear around the same frame the previous one
    # disapeard and remain for atleast 500 more frames or untill the end
    
    while filtered_tuples != []:
        
#     Step 2
    
        begin_ids = list(set(df['id']))

        # Given list of IDs
        given_ids = begin_ids
    
        not_untill_end = []

        last_frame_appearance = {}

        # Iterate over the IDs and find the last frame where each ID appears
        for id in given_ids:
            last_frame = df[df['id'] == id]['frame'].max()
            last_frame_appearance[id] = last_frame if not pd.isnull(last_frame) else "Does not appear"
            if last_frame != df['frame'].max():
                not_untill_end.append(id)

        id_transitions = find_id_transitions(df, frame_threshold=100)

        filtered_tuples = filter_tuples_by_first_value(id_transitions, not_untill_end)
    
        #     Step 3
        # check if second id in tuple appears 40 frames before last frame of first id in tuple, 
        # if so then probably just two fish close to each other

        filtered_tuples = remove_tuples_with_large_time_gap(filtered_tuples, df)

    #     Step 4
        # Create a dictionary where keys are the second elements and values are lists of corresponding first elements
        second_to_first_dict = {}
        for first, second in filtered_tuples:
            if second not in second_to_first_dict:
                second_to_first_dict[second] = [first]
            else:
                second_to_first_dict[second].append(first)

        # Check if any value has more than one occurrence
        for second, first_list in second_to_first_dict.items():
            if len(first_list) > 1:
                for first in first_list:
                    # Check if the corresponding first element appears in another tuple
                    if any((first, other_second) in filtered_tuples for other_second in second_to_first_dict if other_second != second):

                        # Tuple to remove
                        tuple_to_remove = (first, second)

                        # Remove the tuple from the tuple list
                        filtered_tuples = [pair for pair in filtered_tuples if pair != tuple_to_remove]

    #     Step 5

        ids_to_remove = [x[1] for x in filtered_tuples]

        df = replace_ids_in_dataframe(filtered_tuples, df)
    
    renumbered_df = renumber_ids(df.copy())

    return renumbered_df

In [213]:
print(len(set(Re_ID(df_TR_1)['id'])))
df_129_1 = (Re_ID(df_TR_1))
display(df_129_1)
# print(len(set(Re_ID(df_TR_2)['id'])))
# df_129_2 = (Re_ID(df_TR_2))
# display(df_129_2)
# print(len(set(Re_ID(df_TR_349_1)['id'])))
# df_349_1 = (Re_ID(df_TR_349_1))
# display(df_349_1)
# print(len(set(Re_ID(df_TR_349_2)['id'])))
# df_349_2 = (Re_ID(df_TR_349_2))
# display(df_349_2)

# df_TR_1_min = (Re_ID(df_TR_1_min))
# print(len(set(df_TR_1_min['id'])))
# display(df_TR_1_min)

16


,frame,id,x,y,x-offset,y-offset
0,0,0,1226.226318,423.229614,12.112671,7.280457
1,0,1,1495.286865,441.758362,8.967773,10.286316
2,0,2,1205.954102,595.064697,15.199463,5.312988
3,0,3,1078.275879,504.836670,10.427490,5.429565
4,0,4,1222.945312,323.671387,11.825439,5.108154
...,...,...,...,...,...,...
30213,3116,5,1044.965637,260.698135,7.096985,14.554489
30214,3116,4,1304.644287,187.841652,16.015503,6.733131
30215,3116,6,1312.471375,230.199074,6.553772,12.596809
30216,3116,14,1370.705322,649.070770,21.828003,18.922699


In [214]:
def tuple_differences(tuple1, tuple2):
    if len(tuple1) != len(tuple2):
        raise ValueError("Tuples must have the same length")
    
    return tuple(x - y for x, y in zip(tuple1, tuple2))

# check re-id performance
def Check_Re_ID_Performance(df,df_GT):
    correct_id = 0
    wrong_id = 0
    too_many_fish = 0
    too_little_fish = 0
    for frame in set(df_GT['frame']):
        for index, row in df_GT.loc[df_GT['frame']==frame].iterrows():
            for index2, row2 in df.loc[df['frame']==frame].iterrows(): 
                dif = tuple_differences((row['x'],row['y']),(row2['x'],row2['y']))
                if (abs(dif[0]) + abs(dif[1])) < 10:
                    correct_id += 1

        if len(df.loc[df['frame']==frame]) > len(df_GT.loc[df_GT['frame']==frame]):
            too_many_fish += 1
        if len(df.loc[df['frame']==frame]) < len(df_GT.loc[df_GT['frame']==frame]):
            too_little_fish += 1

    print("Percentage correctly IDed fish:",(correct_id/len(df_GT)) * 100,"%")
    print("Percentage wrongly IDed fish:",((len(df_GT) - correct_id)/len(df_GT)) * 100,"%")
    print("Percentage Frames with too many fish:",(too_many_fish/len(set(df_GT['frame'])))*100,"%")
    print("Percentage Frames with too little fish:",(too_little_fish/len(set(df_GT['frame'])))*100,"%")
    print("Amount unique IDs GT:",len(set(df_GT['id'])))
    print("Amount unique IDs Re-ID:",len(set(df['id'])))

print("129_1")    
Check_Re_ID_Performance(df_129_1,df_GT_1)
print('\n')
# print("129_2")    
# Check_Re_ID_Performance(df_129_2,df_GT_2)
# print('\n')
# print("349_1")    
# Check_Re_ID_Performance(df_349_1,df_GT_349_1)
# print('\n')
# print("349_2")    
# Check_Re_ID_Performance(df_349_2,df_GT_349_2)


# print("129_1_min")    
# Check_Re_ID_Performance(df_TR_1_min,df_GT_1)
# print('\n')


129_1
Percentage correctly IDed fish: 0.897321280752293 %
Percentage wrongly IDed fish: 99.1026787192477 %
Percentage Frames with too many fish: 2.438241899262111 %
Percentage Frames with too little fish: 5.646454924606994 %
Amount unique IDs GT: 15
Amount unique IDs Re-ID: 16


129_1_min
Percentage correctly IDed fish: 0.5496506738187478 %
Percentage wrongly IDed fish: 99.45034932618125 %
Percentage Frames with too many fish: 17.19602181584857 %
Percentage Frames with too little fish: 4.491498235482836 %
Amount unique IDs GT: 15
Amount unique IDs Re-ID: 32




when taking xmin,ymin as x-centre and y-centre 99% accuracy, when taking centre x which is calculated by (xmax + xmax)/2 and centre y which is calculated by (ymin + ymax) /2. accuracy is 0.98% with a 10pixel margin

In [215]:
display(df_129_2)

,frame,id,x,y,x-offset,y-offset
0,0,0,1160.961548,345.542725,10.602417,7.335327
1,0,1,1117.533447,447.785370,13.001953,5.743134
2,0,2,1056.066162,444.945923,17.189209,6.576630
3,0,3,1456.922363,472.932953,9.117432,10.052765
4,0,4,1019.735474,361.477234,10.361694,4.094391
...,...,...,...,...,...,...
30605,3117,2,1206.343994,200.705490,16.656860,8.054703
30606,3117,13,1323.102539,688.727936,21.601074,21.305084
30607,3117,4,942.183563,278.734467,6.434113,12.406189
30608,3117,7,934.497375,427.175964,10.431396,4.073364


In [216]:
# Save DataFrame to CSV
df_129_1.to_csv('df_129_1_Re_ID.csv', index=False)
df_129_2.to_csv('df_129_2_Re_ID.csv', index=False)
# df_349_1.to_csv('df_349_1_Re_ID.csv', index=False)
# df_349_2.to_csv('df_349_2_Re_ID.csv', index=False)

In [90]:
print(len(df_129_2.loc[df_129_2['id'] == 0]))
print(len(df_129_2.loc[df_129_2['id'] == 1]))
print(len(df_129_2.loc[df_129_2['id'] == 2]))
print(len(df_129_2.loc[df_129_2['id'] == 3]))
print(len(df_129_2.loc[df_129_2['id'] == 4]))
print(len(df_129_2.loc[df_129_2['id'] == 5]))
print(len(df_129_2.loc[df_129_2['id'] == 6]))
print(len(df_129_2.loc[df_129_2['id'] == 7]))
print(len(df_129_2.loc[df_129_2['id'] == 8]))
print(len(df_129_2.loc[df_129_2['id'] == 9]))
print(len(df_129_2.loc[df_129_2['id'] == 10]))
print(len(df_129_2.loc[df_129_2['id'] == 11]))
print(len(df_129_2.loc[df_129_2['id'] == 12]))
print(len(df_129_2.loc[df_129_2['id'] == 13]))
print(len(df_129_2.loc[df_129_2['id'] == 14]))
print(set(df_129_2['id']))

3118
3096
3096
343
3120
3056
3101
3102
3113
1068
1508
1206
846
547
218
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


In [42]:
print(len(df_349_1.loc[df_349_1['ID'] == 0]))
print(len(df_349_1.loc[df_349_1['ID'] == 1]))
print(len(df_349_1.loc[df_349_1['ID'] == 2]))
print(len(df_349_1.loc[df_349_1['ID'] == 3]))
print(len(df_349_1.loc[df_349_1['ID'] == 4]))
print(len(df_349_1.loc[df_349_1['ID'] == 5]))
print(len(df_349_1.loc[df_349_1['ID'] == 6]))
print(len(df_349_1.loc[df_349_1['ID'] == 7]))
print(len(df_349_1.loc[df_349_1['ID'] == 8]))
print(len(df_349_1.loc[df_349_1['ID'] == 9]))
print(len(df_349_1.loc[df_349_1['ID'] == 10]))
print(set(df_349_1['ID']))

1383
3063
2723
3117
3117
3106
3117
2924
2610
1579
393
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [217]:
most_common_tr_gt = pd.read_csv('most_common_tr_gt.csv')
df_129_1_Re_ID = pd.read_csv('df_129_1_Re_ID.csv')
display(most_common_tr_gt)
display(df_129_1_Re_ID.loc[df_129_1_Re_ID['frame'] == 2890])
display(df_GT_1.loc[df_GT_1['frame'] == 2890])

,tr_id,gt_id
0,0.0,1.0
1,1.0,3.0
2,2.0,9.0
3,3.0,7.0
4,4.0,6.0
5,5.0,8.0
6,6.0,2.0
7,7.0,4.0
8,8.0,5.0
9,9.0,10.0


,frame,id,x,y,x-offset,y-offset
27155,2890,2,1143.118042,575.621857,9.940430,10.430695
27156,2890,7,1051.812744,377.355881,7.515991,5.707962
27157,2890,8,1015.926056,270.480240,11.375946,4.802902
27158,2890,10,1089.907715,288.728104,5.487305,15.689651
27159,2890,11,1156.321411,343.010376,6.398926,5.510315
27160,2890,12,1197.792297,218.729622,8.274475,10.830254
27161,2890,3,993.679749,496.838486,8.057983,11.434250
27162,2890,0,1215.831238,345.434753,10.828186,18.983978
27163,2890,5,1064.712463,335.127350,11.801819,10.326416
27164,2890,4,1314.199829,234.463409,5.238525,13.570847


,frame,id,x,y,x_offset,y_offset
27264,2890,11,1084.76,272.32,12.05,33.39
27265,2890,13,1190.17,208.57,16.10,20.80
27266,2890,12,1149.20,339.98,14.28,8.98
27267,2890,14,1345.10,624.34,33.82,17.80
27268,2890,1,1205.18,326.54,22.83,39.99
27269,2890,2,1309.11,220.08,10.41,32.28
27270,2890,4,1045.75,372.07,13.80,10.33
27271,2890,5,1004.41,265.26,24.71,10.42
27272,2890,6,1304.74,241.52,11.62,31.26
27273,2890,7,985.26,484.96,16.13,23.45


In [218]:
# Create dictionary

gt_id_to_tr_id = dict(zip(most_common_tr_gt['gt_id'], most_common_tr_gt['tr_id']))
print(gt_id_to_tr_id)

gt_id_to_tr_id_REAL = {1:0,3:1, 9:2, 2:6, 7:3, 8:5, 4:7, 5:8, 6:4, 14:14, 15:15, 13:12, 12:11, 11:10, 10:9}
print(gt_id_to_tr_id_REAL)

{1.0: 0.0, 3.0: 1.0, 9.0: 13.0, 7.0: 3.0, 6.0: 4.0, 8.0: 5.0, 2.0: 6.0, 4.0: 7.0, 5.0: 8.0, 10.0: 9.0, 11.0: 10.0, 12.0: 11.0, 13.0: 12.0, 14.0: 14.0, 15.0: 15.0}
{1: 0, 3: 1, 9: 2, 2: 6, 7: 3, 8: 5, 4: 7, 5: 8, 6: 4, 14: 14, 15: 15, 13: 12, 12: 11, 11: 10, 10: 9}


In [219]:
def tuple_differences(tuple1, tuple2):
    if len(tuple1) != len(tuple2):
        raise ValueError("Tuples must have the same length")
    
    return tuple(x - y for x, y in zip(tuple1, tuple2))

# check re-id performance
def Check_Re_ID_Performance_v2(df,df_GT,gt_id_to_tr_id):
    df_perf = pd.DataFrame()
    correct_id = 0
    wrong_id = 0
    too_many_fish = 0
    too_little_fish = 0
    for frame in set(df_GT['frame']):
        if frame in set(df['frame']):
            for index, row in df_GT.loc[df_GT['frame']==frame].iterrows():
                id_match = gt_id_to_tr_id[row['id']]
                match_row = df.loc[(df['frame']==frame) & (df['id']==id_match)]
                if len(match_row) == 1:
#                     print(frame,row['id'],id_match)
#                     print(row)
#                     print(match_row)
#                     print(float(match_row['x']),"XXXXX")
                    gt_x = float(row['x'])
                    gt_y = float(row['y'])
                    gt_x_offset = float(row['x_offset'])
                    gt_y_offset = float(row['y_offset'])
                    pred_x = float(match_row['x'])
                    pred_y = float(match_row['y'])
                    pred_x_offset = float(match_row['x-offset'])
                    pred_y_offset = float(match_row['y-offset'])
                    iou = calculate_iou(gt_x, gt_y, gt_x_offset, gt_y_offset, pred_x, pred_y, pred_x_offset, pred_y_offset)


                    row_dict = {'tr_id':int(match_row['id']) ,'gt_id':int(row['id']), 'iou':iou, 'frame':frame}

                    df_row = pd.DataFrame.from_dict([row_dict])

                    df_perf = pd.concat([df_perf,df_row],ignore_index=True)
            
    return df_perf

iou_df = Check_Re_ID_Performance_v2(df_129_1_Re_ID,df_GT_1,gt_id_to_tr_id)

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2016415472.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_x = float(match_row['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2016415472.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_y = float(match_row['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2016415472.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_x_offset = float(match_row['x-offset'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2016415472.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.

In [220]:
display(iou_df)

,tr_id,gt_id,iou,frame
0,0,1,0.236149,1
1,6,2,0.212581,1
2,1,3,0.264153,1
3,4,6,0.228453,1
4,3,7,0.234905,1
...,...,...,...,...
26930,7,4,0.194010,3116
26931,8,5,0.206210,3116
26932,4,6,0.000000,3116
26933,3,7,0.211144,3116


In [221]:
def calculate_iou(gt_x, gt_y, gt_x_offset, gt_y_offset, pred_x, pred_y, pred_x_offset, pred_y_offset):
    """Calculates Intersection over Union (IoU) for two bounding boxes.

    Args:
        gt_x: X-coordinate of the top-left corner of the ground truth box.
        gt_y: Y-coordinate of the top-left corner of the ground truth box.
        gt_x_offset: Width of the ground truth box.
        gt_y_offset: Height of the ground truth box.
        pred_x: X-coordinate of the top-left corner of the predicted box.
        pred_y: Y-coordinate of the top-left corner of the predicted box.
        pred_x_offset: Width of the predicted box.
        pred_y_offset: Height of the predicted box.

    Returns:
        IoU value (float)
    """

    # Find intersection coordinates
    x_min_intersection = max(gt_x, pred_x)
    y_min_intersection = max(gt_y, pred_y)
    x_max_intersection = min(gt_x + gt_x_offset, pred_x + pred_x_offset)
    y_max_intersection = min(gt_y + gt_y_offset, pred_y + pred_y_offset)

    # Calculate intersection width and height (zero if no overlap)
    width_intersection = max(0, x_max_intersection - x_min_intersection) 
    height_intersection = max(0, y_max_intersection - y_min_intersection)

    # Calculate intersection and union areas
    area_intersection = width_intersection * height_intersection
    area_gt = gt_x_offset * gt_y_offset
    area_pred = pred_x_offset * pred_y_offset
    area_union = area_gt + area_pred - area_intersection

    # Calculate IoU
    IoU = area_intersection / area_union

    return IoU

In [241]:
def Check_Re_ID_Performance_v3(df,df_GT,gt_id_to_tr_id):
    df_perf = pd.DataFrame()
    Correctly_identified_within_margins = 0
    Not_identified_at_all = 0
    Identified_but_not_within_margins = 0
    too_many_fish = 0
    Not_a_real_fish = 0
    for frame in set(df_GT['frame']):
        if frame in set(df['frame']):
            for index, row in df_GT.loc[df_GT['frame']==frame].iterrows():
                id_match = gt_id_to_tr_id[row['id']]
                match_row = df.loc[(df['frame']==frame) & (df['id']==id_match)]
                if len(match_row) == 1:
#                     print(frame,row['id'],id_match)
#                     print(row)
#                     print(match_row)
#                     print(float(match_row['x']),"XXXXX")
                    gt_x = float(row['x'])
                    gt_y = float(row['y'])
                    gt_x_offset = float(row['x_offset'])
                    gt_y_offset = float(row['y_offset'])
                    pred_x = float(match_row['x'])
                    pred_y = float(match_row['y'])
                    pred_x_offset = float(match_row['x-offset'])
                    pred_y_offset = float(match_row['y-offset'])
                    dif_x = abs(abs(gt_x) - abs(pred_x))
                    dif_y = abs(abs(gt_y) - abs(pred_y))
                    tot_dif = dif_x + dif_y
#                     if tot_dif < 10:
                    if (dif_x < 11) & (dif_y < 11):
#                         print(frame,int(row['id']),int(id_match),dif_x,dif_y,tot_dif)
                        Correctly_identified_within_margins += 1
#                     elif tot_dif < 30:
                    elif (dif_x < 22) & (dif_y < 22):
                        Identified_but_not_within_margins += 1
                    else:
                        Not_a_real_fish += 1
                    
                elif len(match_row) == 0:
#                     print(frame)
                    Not_identified_at_all += 1
#                 else:
#                     too_many_fish += 1
            if len(df['frame']) > len(df_GT['frame']):
                dif = len(df['frame']) - len(df_GT['frame'])
#                 print(len(df['frame']),len(df_GT['frame']))
                too_many_fish += dif
                    
    return[Correctly_identified_within_margins,Identified_but_not_within_margins,Not_a_real_fish,Not_identified_at_all,too_many_fish]

res = Check_Re_ID_Performance_v3(df_129_1_Re_ID,df_GT_1,gt_id_to_tr_id_REAL)
# res2 = Check_Re_ID_Performance_v3(df_TR_1_min,df_GT_1,gt_id_to_tr_id)



/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2799356712.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_x = float(match_row['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2799356712.py:23: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_y = float(match_row['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2799356712.py:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_x_offset = float(match_row['x-offset'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_42531/2799356712.py:25: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.

In [242]:
print(res[0]/len(df_GT_1),"fish correclty ided within margin")
print(res[1]/len(df_GT_1), "fish correctly ided not within margen")
print(res[3]/len(df_GT_1), "fish not ided")
print(res[2]/len(df_GT_1), "not a real fish")
print((res[0]/len(df_GT_1)) + (res[1]/len(df_GT_1)) + (res[3]/len(df_GT_1)) + (res[2]/len(df_GT_1)))
precision = res[0] / (res[0] + res[2])
print("Precision:",precision)
# print(res[0],res[2],res[1],res[3])
recall = res[0] / (res[0] + res[1] + res[3])
print("Recall:",recall)
f1 = (2 * precision * recall) / (precision + recall)
print("F1:",f1)

0.3859143736962352 fish correclty ided within margin
0.5879275520678123 fish correctly ided not within margen
0.006158736465679944 fish not ided
0.01953577696102778 not a real fish
0.9995364391907554
Precision: 0.9518170681910985
Recall: 0.3937899111396425
F1: 0.5570957411213613


In [240]:
# Calculate width and height of bounding box
df_GT_1["width"] = df_GT_1["x_offset"] + 2  # 2 is added to account for offset on both sides
df_GT_1["height"] = df_GT_1["y_offset"] + 2

# Calculate average bounding box size
average_bbox_size = df_GT_1[["width", "height"]].mean()

# Print the average bounding box size
print(average_bbox_size)

width     23.338077
height    22.046879
dtype: float64
